In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1116193,2021-03-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1116194,2021-03-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1116195,2021-03-26,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1116196,2021-03-27,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
30055,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30057,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30059,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30061,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30063,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
689228,2021-03-24,Arkansas,Arkansas,5001,2031,35.00,5000,Arkansas,AR,Arkansas,State,3017804
689230,2021-03-25,Arkansas,Arkansas,5001,2068,36.00,5000,Arkansas,AR,Arkansas,State,3017804
689232,2021-03-26,Arkansas,Arkansas,5001,2071,36.00,5000,Arkansas,AR,Arkansas,State,3017804
689234,2021-03-27,Arkansas,Arkansas,5001,2072,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1116193,2021-03-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1116194,2021-03-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1116195,2021-03-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1116196,2021-03-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-03-28') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
432,2021-03-28,Snohomish,Washington,32426,558.00,53061,WA,County,822083,3944.37,67.88
862,2021-03-28,Cook,Illinois,495050,10190.00,17031,IL,County,5150233,9612.19,197.86
1291,2021-03-28,Orange,California,265787,4725.00,6059,CA,County,3175692,8369.42,148.79
1719,2021-03-28,Maricopa,Arizona,523424,9644.00,4013,AZ,County,4485414,11669.47,215.01
2147,2021-03-28,Los Angeles,California,1218879,23103.00,6037,CA,County,10039107,12141.31,230.13
...,...,...,...,...,...,...,...,...,...,...,...
1115653,2021-03-28,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08
1115820,2021-03-28,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1115956,2021-03-28,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1116088,2021-03-28,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
432,2021-03-28,Snohomish,Washington,32426,558.00,53061,WA,County,822083,3944.37,67.88,67.88,3944.37
862,2021-03-28,Cook,Illinois,495050,10190.00,17031,IL,County,5150233,9612.19,197.86,197.86,9612.19
1291,2021-03-28,Orange,California,265787,4725.00,6059,CA,County,3175692,8369.42,148.79,148.79,8369.42
1719,2021-03-28,Maricopa,Arizona,523424,9644.00,4013,AZ,County,4485414,11669.47,215.01,215.01,11669.47
2147,2021-03-28,Los Angeles,California,1218879,23103.00,6037,CA,County,10039107,12141.31,230.13,230.13,12141.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115653,2021-03-28,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08,75.08,1876.88
1115820,2021-03-28,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1115956,2021-03-28,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1116088,2021-03-28,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
720623,2021-03-28,Hale,Alabama,2159,73.00,1065,AL,County,14651,14736.20,498.26,498.26,14736.20,1
784243,2021-03-28,Clarke,Alabama,3462,58.00,1025,AL,County,23622,14655.83,245.53,245.53,14655.83,2
564081,2021-03-28,Lowndes,Alabama,1359,53.00,1085,AL,County,9726,13972.86,544.93,544.93,13972.86,3
419886,2021-03-28,Franklin,Alabama,4184,82.00,1059,AL,County,31362,13340.99,261.46,261.46,13340.99,4
500250,2021-03-28,Etowah,Alabama,13633,342.00,1055,AL,County,102268,13330.66,334.42,334.42,13330.66,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687775,2021-03-28,Converse,Wyoming,982,17.00,56009,WY,County,13822,7104.62,122.99,122.99,7104.62,19
688141,2021-03-28,Sublette,Wyoming,693,7.00,56035,WY,County,9831,7049.13,71.20,71.20,7049.13,20
854640,2021-03-28,Lincoln,Wyoming,1328,12.00,56023,WY,County,19830,6696.92,60.51,60.51,6696.92,21
950846,2021-03-28,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
564081,2021-03-28,Lowndes,Alabama,1359,53.00,1085,AL,County,9726,13972.86,544.93,544.93,13972.86,3,1
720623,2021-03-28,Hale,Alabama,2159,73.00,1065,AL,County,14651,14736.20,498.26,498.26,14736.20,1,2
240193,2021-03-28,Walker,Alabama,7043,274.00,1127,AL,County,63521,11087.67,431.35,431.35,11087.67,28,3
608238,2021-03-28,Greene,Alabama,903,34.00,1063,AL,County,8111,11133.03,419.18,419.18,11133.03,26,4
563345,2021-03-28,Crenshaw,Alabama,1501,57.00,1041,AL,County,13772,10898.93,413.88,413.88,10898.93,29,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606769,2021-03-28,Hot Springs,Wyoming,363,3.00,56017,WY,County,4413,8225.70,67.98,67.98,8225.70,17,19
854640,2021-03-28,Lincoln,Wyoming,1328,12.00,56023,WY,County,19830,6696.92,60.51,60.51,6696.92,21,20
833400,2021-03-28,Uinta,Wyoming,2119,12.00,56041,WY,County,20226,10476.61,59.33,59.33,10476.61,4,21
239068,2021-03-28,Teton,Wyoming,3562,9.00,56039,WY,County,23464,15180.70,38.36,38.36,15180.70,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3944.37,15,22
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3944.37,15,22
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3944.37,15,22
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3944.37,15,22
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3944.37,15,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1113568,2021-03-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1113569,2021-03-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1113570,2021-03-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1113571,2021-03-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
717635,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14736.20,2,1,1.00
717636,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14736.20,2,1,0.00
717637,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14736.20,2,1,0.00
717638,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14736.20,2,1,0.00
717639,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,498.26,14736.20,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868884,2021-03-24,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00
868885,2021-03-25,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00
868886,2021-03-26,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00
868887,2021-03-27,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5564.35,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
561821,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13972.86,1,3,1.00,0.00
561822,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13972.86,1,3,0.00,0.00
561823,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13972.86,1,3,0.00,0.00
561824,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13972.86,1,3,0.00,0.00
561825,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13972.86,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559976,2021-03-24,Albany,Wyoming,3952,11.00,56001,WY,County,38880,10164.61,28.29,28.29,10187.76,23,7,1.00,0.00
559977,2021-03-25,Albany,Wyoming,3952,11.00,56001,WY,County,38880,10164.61,28.29,28.29,10187.76,23,7,0.00,0.00
559978,2021-03-26,Albany,Wyoming,3961,11.00,56001,WY,County,38880,10187.76,28.29,28.29,10187.76,23,7,9.00,0.00
559979,2021-03-27,Albany,Wyoming,3961,11.00,56001,WY,County,38880,10187.76,28.29,28.29,10187.76,23,7,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-03-28') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
286661,2021-03-28,Imperial,California,27406,711.00,6025,CA,County,181215,15123.47,392.35,392.35,15123.47,1,2,32.00,0.00
2147,2021-03-28,Los Angeles,California,1218879,23103.00,6037,CA,County,10039107,12141.31,230.13,230.13,12141.31,2,5,520.00,23.00
142869,2021-03-28,San Bernardino,California,290630,4325.00,6071,CA,County,2180085,13331.13,198.39,198.39,13331.13,3,4,125.00,37.00
501678,2021-03-28,Inyo,California,1372,35.00,6027,CA,County,18039,7605.74,194.02,194.02,7605.74,4,24,3.00,0.00
63949,2021-03-28,Stanislaus,California,58602,992.00,6099,CA,County,550660,10642.14,180.15,180.15,10642.14,5,9,0.00,0.00
79266,2021-03-28,Tulare,California,48960,813.00,6107,CA,County,466195,10502.04,174.39,174.39,10502.04,6,10,0.00,0.00
29667,2021-03-28,Riverside,California,293831,4248.00,6065,CA,County,2470546,11893.36,171.95,171.95,11893.36,7,6,0.00,0.00
50895,2021-03-28,San Joaquin,California,69448,1250.00,6077,CA,County,762148,9112.14,164.01,164.01,9112.14,8,19,0.00,0.00
28893,2021-03-28,Fresno,California,98901,1606.00,6019,CA,County,999101,9899.00,160.74,160.74,9899.00,9,13,170.00,5.00
608180,2021-03-28,Kings,California,22600,241.00,6031,CA,County,152940,14777.04,157.58,157.58,14777.04,10,3,12.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1059298,2021-03-28,Lassen,California,5649,1.00,6035,CA,County,30573,18477.09,3.27,3.27,18477.09,56,1,0.00,0.00
286661,2021-03-28,Imperial,California,27406,711.00,6025,CA,County,181215,15123.47,392.35,392.35,15123.47,1,2,32.00,0.00
608180,2021-03-28,Kings,California,22600,241.00,6031,CA,County,152940,14777.04,157.58,157.58,14777.04,10,3,12.00,0.00
142869,2021-03-28,San Bernardino,California,290630,4325.00,6071,CA,County,2180085,13331.13,198.39,198.39,13331.13,3,4,125.00,37.00
2147,2021-03-28,Los Angeles,California,1218879,23103.00,6037,CA,County,10039107,12141.31,230.13,230.13,12141.31,2,5,520.00,23.00
29667,2021-03-28,Riverside,California,293831,4248.00,6065,CA,County,2470546,11893.36,171.95,171.95,11893.36,7,6,0.00,0.00
179577,2021-03-28,Kern,California,106074,1217.00,6029,CA,County,900202,11783.36,135.19,135.19,11783.36,14,7,80.00,6.00
454705,2021-03-28,Merced,California,30419,434.00,6047,CA,County,277680,10954.70,156.30,156.30,10954.70,11,8,0.00,0.00
63949,2021-03-28,Stanislaus,California,58602,992.00,6099,CA,County,550660,10642.14,180.15,180.15,10642.14,5,9,0.00,0.00
79266,2021-03-28,Tulare,California,48960,813.00,6107,CA,County,466195,10502.04,174.39,174.39,10502.04,6,10,0.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
286661,2021-03-28,Imperial,California,27406,711.00,6025,CA,County,181215,15123.47,392.35,392.35,15123.47,1,2,32.00,0.00
2147,2021-03-28,Los Angeles,California,1218879,23103.00,6037,CA,County,10039107,12141.31,230.13,230.13,12141.31,2,5,520.00,23.00
142869,2021-03-28,San Bernardino,California,290630,4325.00,6071,CA,County,2180085,13331.13,198.39,198.39,13331.13,3,4,125.00,37.00
501678,2021-03-28,Inyo,California,1372,35.00,6027,CA,County,18039,7605.74,194.02,194.02,7605.74,4,24,3.00,0.00
63949,2021-03-28,Stanislaus,California,58602,992.00,6099,CA,County,550660,10642.14,180.15,180.15,10642.14,5,9,0.00,0.00
79266,2021-03-28,Tulare,California,48960,813.00,6107,CA,County,466195,10502.04,174.39,174.39,10502.04,6,10,0.00,0.00
29667,2021-03-28,Riverside,California,293831,4248.00,6065,CA,County,2470546,11893.36,171.95,171.95,11893.36,7,6,0.00,0.00
50895,2021-03-28,San Joaquin,California,69448,1250.00,6077,CA,County,762148,9112.14,164.01,164.01,9112.14,8,19,0.00,0.00
28893,2021-03-28,Fresno,California,98901,1606.00,6019,CA,County,999101,9899.00,160.74,160.74,9899.00,9,13,170.00,5.00
608180,2021-03-28,Kings,California,22600,241.00,6031,CA,County,152940,14777.04,157.58,157.58,14777.04,10,3,12.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15123.47,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15123.47,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15123.47,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15123.47,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15123.47,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3752,03/24/21,Lassen,5637,1.00,18437.84,3.27,3.27,18477.09,56,1,0.00,0.00
3753,03/25/21,Lassen,5637,1.00,18437.84,3.27,3.27,18477.09,56,1,0.00,0.00
3754,03/26/21,Lassen,5649,1.00,18477.09,3.27,3.27,18477.09,56,1,12.00,0.00
3755,03/27/21,Lassen,5649,1.00,18477.09,3.27,3.27,18477.09,56,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15123.47,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15123.47,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15123.47,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15123.47,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15123.47,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3835,03/24/21,Kings,22546,239.00,14741.73,156.27,157.58,14777.04,10,3,14.00,0.00
3836,03/25/21,Kings,22561,239.00,14751.54,156.27,157.58,14777.04,10,3,15.00,0.00
3837,03/26/21,Kings,22578,241.00,14762.65,157.58,157.58,14777.04,10,3,17.00,2.00
3838,03/27/21,Kings,22588,241.00,14769.19,157.58,157.58,14777.04,10,3,10.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)